In [29]:
import json
import graphene
from sagas.ofbiz.schema_testing_g import *
from sagas.ofbiz.connector import OfbizConnector
from sagas.ofbiz.finder import Finder
from sagas.ofbiz.util import QueryHelper
from py4j.java_gateway import java_import

oc=OfbizConnector()
finder=Finder(oc)
helper=QueryHelper(oc, finder)

java_import(oc.j, 'org.apache.ofbiz.entity.util.*')

class Query(graphene.ObjectType):
    movies = graphene.List(lambda: SaMovie, limit=graphene.Int(),
        offset=graphene.Int())
    movie_genres = graphene.List(lambda: SaMovieGenres)

    def resolve_movies(self, info, limit=None, offset=None, **kwargs):
        entity_name = "SaMovie"
        # recs = oc.all(entity_name)
        # print("total record", len(recs))
        findOptions = oc.j.EntityFindOptions()
        if limit is None:
            limit=5
        if offset is None:
            offset=0
        
        # print(limit, offset)
        findOptions.setLimit(limit)
        findOptions.setOffset(offset)
        recs=oc.delegator.findList("SaMovie", None, None, None, findOptions, False)

        ent = oc.delegator.getModelEntity(entity_name)
        result = helper.fill_records(ent, SaMovie, recs)
        return result

    def resolve_movie_genres(self, info):
        entity_name = "SaMovieGenres"
        recs = oc.all(entity_name)
        ent = oc.delegator.getModelEntity(entity_name)
        result = helper.fill_records(ent, SaMovieGenres, recs)
        return result


schema = graphene.Schema(query=Query)

In [30]:
q1 = '''
{
  movies (limit:3, offset:2) {
    title
    saMovieGenresAppl{
        movieGenresId
    }
  }
}
'''.strip()
q2 = '''
{
  movies {
    title
    saMovieGenresAppl{
        movieGenresId
    }
  }
}
'''.strip()

result = schema.execute(q1)
print(json.dumps(result.data, indent=2, ensure_ascii=False))

{
  "movies": [
    {
      "title": "Batman v Superman: Dawn of Justice",
      "saMovieGenresAppl": [
        {
          "movieGenresId": "12"
        },
        {
          "movieGenresId": "14"
        },
        {
          "movieGenresId": "28"
        }
      ]
    },
    {
      "title": "Deadpool",
      "saMovieGenresAppl": [
        {
          "movieGenresId": "12"
        },
        {
          "movieGenresId": "28"
        },
        {
          "movieGenresId": "35"
        }
      ]
    },
    {
      "title": "Doctor Strange",
      "saMovieGenresAppl": [
        {
          "movieGenresId": "12"
        },
        {
          "movieGenresId": "14"
        },
        {
          "movieGenresId": "28"
        },
        {
          "movieGenresId": "878"
        }
      ]
    }
  ]
}


In [11]:
inputs=oc.jmap(title="Deadpool")
finder.find("SaMovie", inputs)

(True,
 [{'overview': 'Deadpool tells the origin story of former Special Forces operative turned mercenary Wade Wilson, who after being subjected to a rogue experiment that leaves him with accelerated healing powers, adopts the alter ego Deadpool. Armed with his new abilit..', 'lastUpdatedStamp': JavaObject id=o3338, 'releaseDate': JavaObject id=o3339, 'createdTxStamp': JavaObject id=o3340, 'createdStamp': JavaObject id=o3341, 'lastUpdatedTxStamp': JavaObject id=o3342, 'movieId': '293660', 'video': 'N', 'title': 'Deadpool', 'originalLanguage': None, 'originalTitle': None, 'popularity': 36.61, 'voteCount': 18642, 'backdropPath': None, 'posterPath': '/inVq3FRqcYIRl2la8iZikYYxFNR.jpg'}])

In [15]:
from py4j.java_gateway import java_import
java_import(oc.j, 'org.apache.ofbiz.entity.util.*')
findOptions = oc.j.EntityFindOptions()
findOptions.setLimit(2)
findOptions.setOffset(2)
rows=oc.delegator.findList("SaMovie", None, None, None, findOptions, False)
for row in rows:
    print(row.get("movieId"), row.get("title"))

209112 Batman v Superman: Dawn of Justice
293660 Deadpool


In [17]:
findOptions = oc.j.EntityFindOptions()
findOptions.setLimit(2)
findOptions.setOffset(3)
rows=oc.delegator.findList("SaMovie", None, None, None, findOptions, False)
for row in rows:
    print(row.get("movieId"), row.get("title"))

293660 Deadpool
284052 Doctor Strange
